In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd

In [6]:
# Load the saved model
model = load_model("model.h5")

# load the encoder
with open("gender_encoder.pkl","rb") as file:
    gen_encoder = pickle.load(file)

with open("geo_encoder.pkl","rb") as file:
    geo_encoder = pickle.load(file)

with open("Scaler.pkl","rb") as file:
    scaler = pickle.load(file)

In [39]:
# Ex 
test_data = pd.DataFrame({
    "CreditScore": 720,
    "Geography": "France",
    "Gender": "Female",
    "Age": 35,
    "Tenure": 5,
    "Balance": 75000.0,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 60000.0
}, index=[0])
test_data


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,720,France,Female,35,5,75000.0,2,1,1,60000.0


In [41]:
# One-hot encode categorical features
geo_encoded = geo_encoder.transform([test_data["Geography"]]).reshape(1,-1)
geo_encoded_df = pd.DataFrame(geo_encoded, columns=geo_encoder.get_feature_names_out(["Geography"]))

geo_encoded_df

c:\Users\KUNAL\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [43]:
test_data["Gender"] = gen_encoder.transform([test_data["Gender"]]).reshape(1,-1)
test_data

c:\Users\KUNAL\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,720,France,0,35,5,75000.0,2,1,1,60000.0


In [44]:
test_data = pd.concat([test_data, geo_encoded_df], axis=1)

In [48]:
test_data = test_data.drop(columns=["Geography"])

In [49]:
# scale the numerical features
test_data = scaler.transform(test_data)

In [50]:
test_data

array([[ 0.70934256, -1.09499335, -0.37056859, -0.00134472, -0.01764635,
         0.80843615,  0.64920267,  0.97481699, -0.70296551,  1.00150113,
        -0.57946723, -0.57638802]])

In [51]:
# Prredict the churn probability
y_pred = model.predict(test_data)
y_pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step


array([[0.02565934]], dtype=float32)

In [53]:
prediction_probability = y_pred[0][0]
if prediction_probability > 0.5:
    prediction = "Churn"
else:
    prediction = "Not Churn"
print(f"Prediction: {prediction} with probability {prediction_probability:.2f}")

Prediction: Not Churn with probability 0.03


In [58]:
gen_encoder.classes_

array(['Female', 'Male'], dtype=object)

In [61]:
gen_encoder.transform(["Female"])

array([0])